In [35]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [36]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [37]:
tf.keras.backend.clear_session()

In [38]:
subject = 'Amapá - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 -30

In [39]:
data = pd.read_csv('2003_mo_model_input_AP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Amapá - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego,Amapá - Consumo de Cimento (t)
0,2003-1,0.711421,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.035813e+06,356591.003430,10.883143,7.033593e+06,8.514392,10.392
1,2003-2,0.711553,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.046432e+06,356647.711004,10.885206,7.037356e+06,8.508753,6.857
2,2003-3,0.711685,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.057052e+06,356704.418579,10.887268,7.041120e+06,8.503114,7.011
3,2003-4,0.711817,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.067671e+06,356761.126153,10.889331,7.044883e+06,8.497475,7.122
4,2003-5,0.711949,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.078290e+06,356817.833728,10.891394,7.048646e+06,8.491835,5.267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.208
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.476
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.236
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.549


In [40]:
input_data = data.iloc[:split_index + 1,1:-1]
# mean = np.mean(input_data, axis=0)
# stddev =  np.std(input_data, axis=0)
# input_data = ((input_data - mean) /stddev)
scaler=MinMaxScaler()
scaled_data = scaler.fit_transform(input_data)
input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,Amapá - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego
0,0.000000,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,0.000000,0.377078,0.459772,0.000000,0.031250
1,0.016604,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,0.006441,0.379481,0.472574,0.010515,0.030561
2,0.033208,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,0.012882,0.381885,0.485377,0.021030,0.029871
3,0.049812,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,0.019322,0.384288,0.498179,0.031545,0.029182
4,0.066416,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,0.025763,0.386691,0.510981,0.042060,0.028493
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.524987,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.968856,0.047639,0.017632,0.956972,0.901730
188,0.502010,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.965298,0.050101,0.013224,0.952114,0.898637
189,0.479033,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.961740,0.052562,0.008816,0.947256,0.895544
190,0.456057,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.958182,0.055024,0.004408,0.942399,0.892451


In [41]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      10.000
1       6.136
2       5.122
3       3.926
4       6.197
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Amapá - Consumo de Cimento (t), Length: 240, dtype: float64

In [42]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Amapá - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego
0,0.000000,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,0.000000,0.377078,0.459772,0.000000,0.031250
1,0.016604,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,0.006441,0.379481,0.472574,0.010515,0.030561
2,0.033208,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,0.012882,0.381885,0.485377,0.021030,0.029871
3,0.049812,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,0.019322,0.384288,0.498179,0.031545,0.029182
4,0.066416,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,0.025763,0.386691,0.510981,0.042060,0.028493
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.987023,0.379453,0.024030,0.338208,0.039971,0.890665,0.956993,0.182507,0.980118,0.093565,0.049927,0.968794,0.996442
158,0.974046,0.332832,0.021531,0.371581,0.027548,0.893949,0.953662,0.168812,0.981925,0.085059,0.051900,0.971631,0.992884
159,0.961069,0.313954,0.019385,0.365349,0.037038,0.897232,0.950331,0.152959,0.983733,0.076553,0.053873,0.974468,0.989327
160,0.948092,0.294856,0.017671,0.404993,0.070841,0.900516,0.947000,0.137613,0.985540,0.068047,0.055846,0.977305,0.985769


In [43]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      10.000
1       6.136
2       5.122
3       3.926
4       6.197
        ...  
157     8.322
158    11.373
159     9.651
160    10.855
161    13.970
Name: Amapá - Consumo de Cimento (t), Length: 162, dtype: float64

In [44]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [45]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 13)

In [46]:
# input para treinamento
test_input = input_data.iloc[train_split-window_size:split_index + 1]
test_input

,Amapá - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego
126,0.816354,0.612440,0.052718,0.094523,0.201912,0.780557,0.994213,0.372054,0.856454,0.776195,0.439882,0.929165,0.854574
127,0.802872,0.626227,0.056653,0.200555,0.209545,0.784414,0.995177,0.375438,0.861855,0.767179,0.418917,0.929945,0.861253
128,0.789390,0.645442,0.060771,0.214993,0.238038,0.788271,0.996142,0.372254,0.867255,0.758162,0.397953,0.930726,0.867931
129,0.775907,0.673151,0.065105,0.053026,0.249843,0.792128,0.997106,0.369458,0.872655,0.749146,0.376989,0.931506,0.874609
130,0.762425,0.699949,0.069099,0.016760,0.266770,0.795984,0.998071,0.371547,0.878056,0.740129,0.356024,0.932287,0.881288
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.524987,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.968856,0.047639,0.017632,0.956972,0.901730
188,0.502010,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.965298,0.050101,0.013224,0.952114,0.898637
189,0.479033,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.961740,0.052562,0.008816,0.947256,0.895544
190,0.456057,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.958182,0.055024,0.004408,0.942399,0.892451


In [47]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test_target

array([12.399, 11.268,  8.741, 12.358, 11.876, 11.713, 11.449,  7.805,
        7.726,  8.516,  8.503, 10.175, 11.775,  9.724, 10.069, 13.93 ,
       11.918, 11.757,  9.691,  7.378,  6.97 ,  7.612,  8.442,  8.835,
       11.251, 11.342, 13.11 , 13.195, 10.053, 12.274])

In [48]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [49]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
#     optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(36, activation='tanh', 
                                 return_sequences=False), 
                                 input_shape=(train_input.shape[1], 
                                              train_input.shape[2])),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(180, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [50]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(20)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [51]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[4013962441, 3412520494, 2497844168, 1170259549, 3581653399, 508604672, 2676102043, 1672478936, 3699799400, 2563583883, 1467837276, 1564096472, 1961931955, 2077073841, 406885189, 2965592059, 914530359, 2605495783, 1966978190, 3962811231]


Step: 0 ___________________________________________
val_loss: 6.750421524047852
winner_seed: 4013962441


Step: 1 ___________________________________________
val_loss: 6.737621307373047
winner_seed: 3412520494


Step: 2 ___________________________________________
val_loss: 6.985787868499756


Step: 3 ___________________________________________
val_loss: 6.850013732910156


Step: 4 ___________________________________________
val_loss: 6.372694492340088
winner_seed: 3581653399


Step: 5 ___________________________________________
val_loss: 7.18047571182251


Step: 6 ___________________________________________
val_loss: 5.700832366943359
winner_seed: 2676102043


Step: 7 ___________________________________________
val_loss: 7.712840557098389


Step: 8 _

In [52]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 131ms/step - loss: 244.0505 - val_loss: 264.8690
Epoch 2/10000
4/4 [==============================] - 0s 11ms/step - loss: 226.9406 - val_loss: 246.9614
Epoch 3/10000
4/4 [==============================] - 0s 11ms/step - loss: 210.1157 - val_loss: 228.7532
Epoch 4/10000
4/4 [==============================] - 0s 11ms/step - loss: 192.8884 - val_loss: 209.8186
Epoch 5/10000
4/4 [==============================] - 0s 9ms/step - loss: 174.4368 - val_loss: 190.1533
Epoch 6/10000
4/4 [==============================] - 0s 10ms/step - loss: 155.9055 - val_loss: 170.5986
Epoch 7/10000
4/4 [==============================] - 0s 10ms/step - loss: 137.4380 - val_loss: 152.9025
Epoch 8/10000
4/4 [==============================] - 0s 10ms/step - loss: 121.7089 - val_loss: 137.5884
Epoch 9/10000
4/4 [==============================] - 0s 10ms/step - loss: 107.9059 - val_loss: 124.1229
Epoch 10/10000
4/4 [==============================] - 0s 10ms/st

4/4 [==============================] - 0s 11ms/step - loss: 14.5454 - val_loss: 22.9664
Epoch 81/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.1943 - val_loss: 22.5060
Epoch 82/10000
4/4 [==============================] - 0s 11ms/step - loss: 14.3179 - val_loss: 22.2521
Epoch 83/10000
4/4 [==============================] - 0s 10ms/step - loss: 13.8238 - val_loss: 21.9654
Epoch 84/10000
4/4 [==============================] - 0s 10ms/step - loss: 13.7639 - val_loss: 21.1190
Epoch 85/10000
4/4 [==============================] - 0s 10ms/step - loss: 13.5635 - val_loss: 21.3922
Epoch 86/10000
4/4 [==============================] - 0s 10ms/step - loss: 13.4078 - val_loss: 21.0132
Epoch 87/10000
4/4 [==============================] - 0s 11ms/step - loss: 13.2462 - val_loss: 20.9391
Epoch 88/10000
4/4 [==============================] - 0s 10ms/step - loss: 13.2965 - val_loss: 20.8809
Epoch 89/10000
4/4 [==============================] - 0s 10ms/step - loss: 13.1763 - val

4/4 [==============================] - 0s 10ms/step - loss: 6.9481 - val_loss: 12.8902
Epoch 160/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.9353 - val_loss: 11.8794
Epoch 161/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.9087 - val_loss: 11.8779
Epoch 162/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.8129 - val_loss: 11.6344
Epoch 163/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.8892 - val_loss: 11.7819
Epoch 164/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.8044 - val_loss: 11.0617
Epoch 165/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.7526 - val_loss: 12.0388
Epoch 166/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.6456 - val_loss: 12.6463
Epoch 167/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.5783 - val_loss: 11.3720
Epoch 168/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.4221 - val_

4/4 [==============================] - 0s 10ms/step - loss: 6.5338 - val_loss: 12.7743
Epoch 239/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.9346 - val_loss: 14.7850
Epoch 240/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.7674 - val_loss: 13.4169
Epoch 241/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.1318 - val_loss: 11.7859
Epoch 242/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.2156 - val_loss: 13.1901
Epoch 243/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.1366 - val_loss: 13.3135
Epoch 244/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.9684 - val_loss: 10.2389
Epoch 245/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.1952 - val_loss: 11.5799
Epoch 246/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.5395 - val_loss: 13.2503
Epoch 247/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.6969 - val_

Epoch 318/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.9685 - val_loss: 12.6327
Epoch 319/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.9575 - val_loss: 10.4197
Epoch 320/10000
4/4 [==============================] - 0s 9ms/step - loss: 4.7880 - val_loss: 10.5036
Epoch 321/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.7268 - val_loss: 11.1117
Epoch 322/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.7886 - val_loss: 10.8300
Epoch 323/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.6587 - val_loss: 10.7862
Epoch 324/10000
4/4 [==============================] - 0s 11ms/step - loss: 4.7217 - val_loss: 10.6034
Epoch 325/10000
4/4 [==============================] - 0s 11ms/step - loss: 4.7736 - val_loss: 10.9852
Epoch 326/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.6230 - val_loss: 11.2788
Epoch 327/10000
4/4 [==============================] - 0s 12ms/step - loss

...

4/4 [==============================] - 0s 11ms/step - loss: 2.1382 - val_loss: 8.8133
Epoch 1907/10000
4/4 [==============================] - 0s 11ms/step - loss: 2.0937 - val_loss: 9.1877
Epoch 1908/10000
4/4 [==============================] - 0s 11ms/step - loss: 2.1177 - val_loss: 8.2669
Epoch 1909/10000
4/4 [==============================] - 0s 10ms/step - loss: 2.6578 - val_loss: 7.4260
Epoch 1910/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.0136 - val_loss: 10.0395
Epoch 1911/10000
4/4 [==============================] - 0s 11ms/step - loss: 3.0509 - val_loss: 9.8950
Epoch 1912/10000
4/4 [==============================] - 0s 10ms/step - loss: 2.3994 - val_loss: 8.6293
Epoch 1913/10000
4/4 [==============================] - 0s 11ms/step - loss: 2.4818 - val_loss: 9.2031
Epoch 1914/10000
4/4 [==============================] - 0s 10ms/step - loss: 2.2942 - val_loss: 8.7840
Epoch 1915/10000
4/4 [==============================] - 0s 10ms/step - loss: 2.2705 - val

4/4 [==============================] - 0s 10ms/step - loss: 2.6605 - val_loss: 9.6951
Epoch 1986/10000
4/4 [==============================] - 0s 10ms/step - loss: 2.4633 - val_loss: 9.4213
Epoch 1987/10000
4/4 [==============================] - 0s 10ms/step - loss: 2.9549 - val_loss: 8.6427
Epoch 1988/10000
4/4 [==============================] - 0s 11ms/step - loss: 2.8594 - val_loss: 9.3003
Epoch 1989/10000
4/4 [==============================] - 0s 11ms/step - loss: 2.6755 - val_loss: 8.9383
Epoch 1990/10000
4/4 [==============================] - 0s 11ms/step - loss: 2.4410 - val_loss: 8.4917
Epoch 1991/10000
4/4 [==============================] - 0s 10ms/step - loss: 2.3857 - val_loss: 9.2514
Epoch 1992/10000
4/4 [==============================] - 0s 10ms/step - loss: 2.4125 - val_loss: 8.4038
Epoch 1993/10000
4/4 [==============================] - 0s 10ms/step - loss: 2.3157 - val_loss: 8.1404
Epoch 1994/10000
4/4 [==============================] - 0s 10ms/step - loss: 2.2647 - val_

4/4 [==============================] - 0s 10ms/step - loss: 6.0168 - val_loss: 13.1949
Epoch 2065/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.8635 - val_loss: 12.5072
Epoch 2066/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.2356 - val_loss: 12.0537
Epoch 2067/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.3893 - val_loss: 11.4103
Epoch 2068/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.3866 - val_loss: 9.1982
Epoch 2069/10000
4/4 [==============================] - 0s 9ms/step - loss: 5.3769 - val_loss: 8.3484
Epoch 2070/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.5065 - val_loss: 9.3253
Epoch 2071/10000
4/4 [==============================] - 0s 11ms/step - loss: 4.9682 - val_loss: 10.4127
Epoch 2072/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.7368 - val_loss: 9.8633
Epoch 2073/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.9914 - 

4/4 [==============================] - 0s 10ms/step - loss: 5.3471 - val_loss: 10.0643
Epoch 2144/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.2073 - val_loss: 8.5325
Epoch 2145/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.0841 - val_loss: 8.8314
Epoch 2146/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.0201 - val_loss: 7.8705
Epoch 2147/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.4791 - val_loss: 10.3468
Epoch 2148/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.1264 - val_loss: 11.2374
Epoch 2149/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.5622 - val_loss: 11.3530
Epoch 2150/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.3932 - val_loss: 9.0552
Epoch 2151/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.4233 - val_loss: 8.5162
Epoch 2152/10000
4/4 [==============================] - 0s 11ms/step - loss: 4.4210 - 

4/4 [==============================] - 0s 10ms/step - loss: 3.5730 - val_loss: 13.2028
Epoch 2223/10000
4/4 [==============================] - 0s 9ms/step - loss: 3.6650 - val_loss: 13.2821
Epoch 2224/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.7061 - val_loss: 11.7088
Epoch 2225/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.7566 - val_loss: 11.5628
Epoch 2226/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.6545 - val_loss: 10.0497
Epoch 2227/10000
4/4 [==============================] - 0s 12ms/step - loss: 3.6181 - val_loss: 8.8366
Epoch 2228/10000
4/4 [==============================] - 0s 11ms/step - loss: 3.6222 - val_loss: 9.2410
Epoch 2229/10000
4/4 [==============================] - 0s 11ms/step - loss: 3.4517 - val_loss: 13.2661
Epoch 2230/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.3911 - val_loss: 9.5463
Epoch 2231/10000
4/4 [==============================] - 0s 11ms/step - loss: 3.8220 -

In [53]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [54]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 21ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,10.17458,10.124062,10.056103,9.950022,9.915434,9.906684,9.866193,9.732161,9.610868,9.440791,9.310955,9.18915,9.111969,8.939456,8.780739,8.69802,8.590183,8.518743,8.419693,8.294443,8.23691,8.134846,8.022357,7.971478,7.906585,7.953382,7.96035,7.995978,7.990567,8.042925
Target,12.399,11.268,8.741,12.358,11.876,11.713,11.449,7.805,7.726,8.516,8.503,10.175,11.775,9.724,10.069,13.93,11.918,11.757,9.691,7.378,6.97,7.612,8.442,8.835,11.251,11.342,13.11,13.195,10.053,12.274
Error,2.224421,1.143938,1.315103,2.407978,1.960567,1.806316,1.582808,1.927161,1.884869,0.924791,0.807955,0.98585,2.663031,0.784544,1.288261,5.23198,3.327817,3.238257,1.271307,0.916443,1.26691,0.522846,0.419643,0.863522,3.344415,3.388618,5.14965,5.199022,2.062434,4.231075


In [55]:
display(mae)
display(mape)

2.138051

0.19420685

In [56]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [57]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[117.27701]] - Target[122.52900000000001]| =  Error: [[5.2519913]]; MAPE:[[0.04286325]]
1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[101.71884]] - Target[118.101]| =  Error: [[16.382156]]; MAPE:[[0.1387131]]
1/1 [==============================] - 0s 15ms/step
Ano-5: |Prediction[[47.849785]] - Target[71.22500000000001]| =  Error: [[23.375214]]; MAPE:[[0.32818833]]


[array([[5.2519913]], dtype=float32),
 array([[16.382156]], dtype=float32),
 array([[23.375214]], dtype=float32)]

15.00312

0.16992156